**Training notebook** [here](https://www.kaggle.com/bumjunkoo/commonlit-run-glue/edit) will take around 40 minutes to run with GPU enabled   
**Fitting notebook** [here](https://www.kaggle.com/bumjunkoo/commonlit-run-glue-fit) will take 1 hour and ~40 minutes to run with GPU enabled

In [ ]:
import pandas as pd
import numpy as np

test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
test_text = test[['excerpt']].copy()
test_text.rename(columns={'excerpt':'text'}, inplace=True)
test_text.to_csv('test_data.csv', index=False)

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("../input/huggingface-roberta/roberta-base")

test_encodes = tokenizer([i for i in test_text.text], max_length=325, truncation=True)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CommonLitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

test_dataset = CommonLitDataset(test_encodes)

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)
test_dataloader = DataLoader(
    test_dataset, batch_size=16, collate_fn=data_collator
)

# for step, batch in enumerate(test_dataloader):
#     print(batch["input_ids"].shape)
#     if step > 5:
#         break

In [ ]:
from transformers import Trainer, TrainingArguments

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = AutoModelForSequenceClassification.from_pretrained('../input/commonlit-run-glue/output')
model.load_state_dict(torch.load("../input/commonlit-run-glue-fit/model.pt"))
model.to(device)
preds = []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        prediction = outputs.logits
        preds.extend(np.array(prediction.cpu().squeeze()))

sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
sample.target = preds
sample.to_csv('submission.csv',index=False)
sample